In [ ]:
import os
import numpy as np
import scipy as sp
from kernel import run_kernel
from hardata import HarData

In [ ]:
class DataGenerator:
    """
    A class for reading and preprocessing text data.
    """

    def __init__(self, path: str, sequence_length: int):
        """
        Initializes a DataReader object with the path to a text file and the desired sequence length.

        Args:
            path (str): The path to the text file.
            sequence_length (int): The length of the sequences that will be fed to the self.
        """
        with open(path) as f:
            # Read the contents of the file
            self.data = f.read()

        # Find all unique characters in the text
        chars = list(set(self.data))

        # Create dictionaries to map characters to indices and vice versa
        self.char_to_idx = {ch: i for (i, ch) in enumerate(chars)}
        self.idx_to_char = {i: ch for (i, ch) in enumerate(chars)}

        # Store the size of the text data and the size of the vocabulary
        self.data_size = len(self.data)
        self.vocab_size = len(chars)

        # Initialize the pointer that will be used to generate sequences
        self.pointer = 0

        # Store the desired sequence length
        self.sequence_length = sequence_length


    def next_batch(self):
        """
        Generates a batch of input and target sequences.

        Returns:
            inputs_one_hot (np.ndarray): A numpy array with shape `(batch_size, vocab_size)` where each row is a one-hot encoded representation of a character in the input sequence.
            targets (list): A list of integers that correspond to the indices of the characters in the target sequence, which is the same as the input sequence shifted by one position to the right.
        """
        input_start = self.pointer
        input_end = self.pointer + self.sequence_length

        # Get the input sequence as a list of integers
        inputs = [self.char_to_idx[ch] for ch in self.data[input_start:input_end]]

        # One-hot encode the input sequence
        inputs_one_hot = np.zeros((len(inputs), self.vocab_size))
#         print('batch_size:', (len(inputs)))
        inputs_one_hot[np.arange(len(inputs)), inputs] = 1

        # Get the target sequence as a list of integers
        targets = [self.char_to_idx[ch] for ch in self.data[input_start + 1:input_end + 1]]

        # Update the pointer
        self.pointer += self.sequence_length

        # Reset the pointer if the next batch would exceed the length of the text data
        if self.pointer + self.sequence_length + 1 >= self.data_size:
            self.pointer = 0

        return inputs_one_hot, targets

In [ ]:
def cut_in_sequences(x,y,seq_len,inc=1):

    sequences_x = []
    sequences_y = []

    for s in range(0,x.shape[0] - seq_len,inc):
        start = s
        end = start+seq_len
        sequences_x.append(x[start:end])
        sequences_y.append(y[start:end])

    return np.stack(sequences_x,axis=1),np.stack(sequences_y,axis=1)


numID = 13


class HarData:

    def __init__(self,seq_len=16):
        print("Parsing for Patient File {}".format(numID))

        train_x = np.loadtxt(f"Train_570_{numID}_glucose.txt")
        train_y = (np.loadtxt(f"Train_570_{numID}_glucose.txt") - 1)  # .astype(np.int32)
        train_ins = np.loadtxt(f"Train_570_{numID}_bolus.txt")
        train_meal = np.loadtxt(f"Train_570_{numID}_meal.txt")
        train_basal = np.loadtxt(f"Train_570_{numID}_basal.txt")
        train_initIsc1 = np.loadtxt(f"Train_570_{numID}_Isc1.txt")
        train_initIsc2 = np.loadtxt(f"Train_570_{numID}_Isc2.txt")
        train_initIp = np.loadtxt(f"Train_570_{numID}_Ip.txt")
        train_BW = np.loadtxt(f"Train_570_{numID}_BW.txt")
        train_u2ss = np.loadtxt(f"Train_570_{numID}_u2ss.txt")

        

        train_meal2 = train_meal

        test_x = np.loadtxt(f"Train_570_{numID}_glucose.txt")
        test_y = np.loadtxt(f"Train_570_{numID}_glucose.txt") - 1  # .astype(np.int32)
        test_ins = np.loadtxt(f"Train_570_{numID}_bolus.txt")
        test_meal = np.loadtxt(f"Train_570_{numID}_meal.txt")
        test_meal2 = test_meal
        test_basal = np.loadtxt(f"Train_570_{numID}_basal.txt")
        test_initIsc1 = np.loadtxt(f"Train_570_{numID}_Isc1.txt")
        test_initIsc2 = np.loadtxt(f"Train_570_{numID}_Isc2.txt")
        test_initIp = np.loadtxt(f"Train_570_{numID}_Ip.txt")
        test_BW = np.loadtxt(f"Train_570_{numID}_BW.txt")
        test_u2ss = np.loadtxt(f"Train_570_{numID}_u2ss.txt")

        
        # shape = tf.shape(test_basal)
        # with tf.compat.v1.Session() as sess:
        #     numpy_number = sess.run(shape[1])
        # Nloop = numpy_number
        # print("Nloop {}".format(Nloop))
        train_x,train_y = cut_in_sequences(train_x,train_y,seq_len)
        train_ins,train_meal = cut_in_sequences(train_ins,train_meal,seq_len)
        train_basal,train_initIsc1 = cut_in_sequences(train_basal,train_initIsc1,seq_len)
        train_initIsc2, train_initIp = cut_in_sequences(train_initIsc2, train_initIp, seq_len)
        train_BW, train_u2ss = cut_in_sequences(train_BW, train_u2ss, seq_len)

        test_x,test_y = cut_in_sequences(test_x,test_y,seq_len,inc=8)
        test_ins,test_meal = cut_in_sequences(test_ins,test_meal,seq_len,inc=8)

        test_basal, test_initIsc1 = cut_in_sequences(test_basal, test_initIsc1, seq_len,inc=8)
        test_initIsc2, test_initIp = cut_in_sequences(test_initIsc2, test_initIp, seq_len,inc=8)
        test_BW, test_u2ss = cut_in_sequences(test_BW, test_u2ss, seq_len,inc=8)
        print("Total number of testing sequences: {}".format(test_initIsc1.shape[1]))
        #permutation = np.random.RandomState(893429).permutation(train_x.shape[1])
        valid_size = int(0.1*train_x.shape[1])
        print("Validation split: {}, training split: {}".format(valid_size,train_x.shape[1]-valid_size))

        self.valid_x = train_x[:,:valid_size]
        self.valid_ins = train_ins[:,:valid_size]
        self.valid_meal = train_meal[:,:valid_size]
        self.valid_y = train_y[:,:valid_size]
        self.valid_basal = train_basal[:,:valid_size]
        self.valid_initIsc1 = train_initIsc1[:,:valid_size]
        self.valid_initIsc2 = train_initIsc2[:, :valid_size]
        self.valid_initIp = train_initIp[:, :valid_size]
        self.valid_BW = train_BW[:, :valid_size]
        self.valid_u2ss = train_u2ss[:, :valid_size]



        self.train_x = train_x[:,valid_size:]
        self.train_ins = train_ins[:,valid_size:]
        self.train_meal = train_meal[:,valid_size:]
        self.train_y = train_y[:,valid_size:]
        self.train_basal = train_basal[:,valid_size:]
        self.train_initIsc1 = train_initIsc1[:, valid_size:]
        self.train_initIsc2 = train_initIsc2[:, valid_size:]
        self.train_initIp = train_initIp[:, valid_size:]
        self.train_BW = train_BW[:, valid_size:]
        self.train_u2ss = train_u2ss[:, valid_size:]

        self.test_x = test_x
        self.test_ins = test_ins
        self.test_meal = test_meal
        self.test_y = test_y
        self.test_basal = test_basal
        self.test_initIsc1 = test_initIsc1
        self.test_initIsc2 = test_initIsc2
        self.test_initIp = test_initIp
        self.test_BW = test_BW
        self.test_u2ss = test_u2ss


In [ ]:
class GRU:
    """
    A class used to represent a Recurrent Neural Network (GRU).

    Attributes
    ----------
    hidden_size : int
        The number of hidden units in the GR.
    vocab_size : int
        The size of the vocabulary used by the GRU.
    sequence_length : int
        The length of the input sequences fed to the GRU.
    self.learning_rate : float
        The learning rate used during training.

    Methods
    -------
    __init__(hidden_size, vocab_size, sequence_length, self.learning_rate)
        Initializes an instance of the GRU class.
    """

    def __init__(self, hidden_size, vocab_size, sequence_length, learning_rate):
        """
        Initializes an instance of the GRU class.

        Parameters
        ----------
        hidden_size : int
            The number of hidden units in the GRU.
        vocab_size : int
            The size of the vocabulary used by the GRU.
        sequence_length : int
            The length of the input sequences fed to the GRU.
        learning_rate : float
            The learning rate used during training.
        """
        # hyper parameters
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.sequence_length = sequence_length
        self.learning_rate = learning_rate

        # model parameters
        self.Wz = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (hidden_size, hidden_size + vocab_size))
        self.bz = np.zeros((hidden_size, 1))

        self.Wr = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (hidden_size, hidden_size + vocab_size))
        self.br = np.zeros((hidden_size, 1))

        self.Wa = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (hidden_size, hidden_size + vocab_size))
        self.ba = np.zeros((hidden_size, 1))

        self.Wy = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (vocab_size, hidden_size))
        self.by = np.zeros((vocab_size, 1))

        # initialize gradients for each parameter
        self.dWz, self.dWr, self.dWa, self.dWy = np.zeros_like(self.Wz), np.zeros_like(self.Wr), np.zeros_like(
            self.Wa), np.zeros_like(self.Wy)
        self.dbz, self.dbr, self.dba, self.dby = np.zeros_like(self.bz), np.zeros_like(self.br), np.zeros_like(
            self.bz), np.zeros_like(self.by)

        # initialize parameters for adamw optimizer
        self.mWz = np.zeros_like(self.Wz)
        self.vWz = np.zeros_like(self.Wz)
        self.mWr = np.zeros_like(self.Wr)
        self.vWr = np.zeros_like(self.Wr)
        self.mWa = np.zeros_like(self.Wa)
        self.vWa = np.zeros_like(self.Wa)
        self.mWy = np.zeros_like(self.Wy)
        self.vWy = np.zeros_like(self.Wy)
        self.mbz = np.zeros_like(self.bz)
        self.vbz = np.zeros_like(self.bz)
        self.mbr = np.zeros_like(self.br)
        self.vbr = np.zeros_like(self.br)
        self.mba = np.zeros_like(self.ba)
        self.vba = np.zeros_like(self.ba)
        self.mby = np.zeros_like(self.by)
        self.vby = np.zeros_like(self.by)

    def sigmoid(self, x):
        """
        Computes the sigmoid activation function for a given input array.

        Parameters:
            x (ndarray): Input array.

        Returns:
            ndarray: Array of the same shape as `x`, containing the sigmoid activation values.
        """
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        """
        Computes the softmax activation function for a given input array.

        Parameters:
            x (ndarray): Input array.

        Returns:
            ndarray: Array of the same shape as `x`, containing the softmax activation values.
        """
        # shift the input to prevent overflow when computing the exponentials
        x = x - np.max(x)
        # compute the exponentials of the shifted input
        p = np.exp(x)
        # normalize the exponentials by dividing by their sum
        return p / np.sum(p)

    def forward(self, X, c_prev, a_prev):
        """
        Performs forward propagation for a simple GRU model.

        Args:
            X (numpy array): Input sequence, shape (sequence_length, input_size)
            c_prev (numpy array): Previous cell state, shape (hidden_size, 1)
            a_prev (numpy array): Previous hidden state, shape (hidden_size, 1)

        Returns: X (numpy array): Input sequence, shape (sequence_length, input_size) c (dictionary): Cell state for
        each time step, keys = time step, values = numpy array shape (hidden_size, 1) r (dictionary): Reset gate for
        each time step, keys = time step, values = numpy array shape (hidden_size, 1) z (dictionary): Update gate for
        each time step, keys = time step, values = numpy array shape (hidden_size, 1) cc (dictionary): Candidate cell
        state for each time step, keys = time step, values = numpy array shape (hidden_size, 1) a (dictionary):
        Hidden state for each time step, keys = time step, values = numpy array shape (hidden_size, 1) y_pred (
        dictionary): Output probability vector for each time step, keys = time step, values = numpy array shape (
        output_size, 1)
        """
        # print('x shape:', X.shape)
        # print('c_prevx shape:', c_prev.shape)
        # print('a_prevx shape:', a_prev.shape)
        # initialize dictionaries for backpropagation
        # initialize dictionaries for backpropagation
        r, z, c, cc, a, y_pred = {}, {}, {}, {}, {}, {}
        c[-1] = np.copy(c_prev)  # store the initial cell state in the dictionary
        a[-1] = np.copy(a_prev)  # store the initial hidden state in the dictionary

        # iterate over each time step in the input sequence
        for t in range(X.shape[0]):
            # concatenate the input and hidden state
            xt = X[t, :].reshape(-1, 1)
            concat = np.vstack((a[t - 1], xt))
            
            wr_update = run_kernel(self.Wr, concat)
            
            # compute the reset gate
            r[t] = self.sigmoid(wr_update + self.br)
            # print('concat:', concat.shape)
            # compute the update gate
            z[t] = self.sigmoid(wr_update + self.bz)

            # compute the candidate cell state
            cc[t] = np.tanh(np.dot(self.Wa, np.vstack((r[t] * a[t - 1], xt))) + self.ba)

            # compute the cell state
            c[t] = z[t] * cc[t] + (1 - z[t]) * c[t - 1]

            # compute the hidden state
            a[t] = c[t]

            
            # compute the output probability vector
            y_pred[t] = self.softmax(np.dot(self.Wy, a[t]) + self.by)
            # print('y_predict:', y_pred[t].shape)
            # print('a[t]:', a[t].shape)
            # print('self.Wy:', self.Wy.shape)
            # print('y_pred[t]:', y_pred[t].shape)

        # return the output probability vectors, cell state, hidden state and gate vectors
        return X, r, z, c, cc, a, y_pred

    def backward(self, X, a_prev, c_prev, r, z, c, cc, a, y_pred, targets):
        """
        Performs backward propagation through time for a GRU network.

        Args:
            X (numpy array): Input sequence, shape (sequence_length, input_size)
            a_prev (numpy array): Previous hidden state, shape (hidden_size, 1)
            r (dictionary): Reset gate for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            z (dictionary): Update gate for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            c (dictionary): Cell state for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            cc (dictionary): Candidate cell state for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            a (dictionary): Hidden state for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            y_pred (dictionary): Output probability vector for each time step, keys = time step, values = numpy array shape (output_size, 1)
            targets (numpy array): Target outputs for each time step, shape (sequence_length, output_size)

        Returns:
            None       
        """
        # Initialize gradients for hidden state
        dc_next = np.zeros_like(c_prev)
        da_next = np.zeros_like(a_prev)

        # Iterate backwards through time steps
        for t in reversed(range(X.shape[0])):
            # compute the gradient of the output probability vector
            dy = np.copy(y_pred[t])
            dy[targets[t]] -= 1

            # compute the gradient of the output layer weights and biases
            self.dWy += np.dot(dy, a[t].T)
            self.dby += dy

            # compute the gradient of the hidden state
            da = np.dot(self.Wy.T, dy) + da_next

            # compute the gradient of the update gate
            xt = X[t, :].reshape(-1, 1)
            concat = np.vstack((a_prev, xt))
            dz = da * (a[t] - c[t])
            self.dWz += np.dot(dz, concat.T)
            self.dbz += dz

            # compute the gradient of the reset gate
            dr = da * np.dot(self.Wz[:, :self.hidden_size].T, dz) * (1 - r[t]) * r[t]
            self.dWr += np.dot(dr, concat.T)
            self.dbr += dr

            # compute the gradient of the current hidden state
            da = np.dot(self.Wa[:, :self.hidden_size].T, dr) + np.dot(self.Wz[:, :self.hidden_size].T, dz)
            self.dWa += np.dot(da * (1 - a[t]**2), concat.T)
            self.dba += da * (1 - a[t]**2)

            # compute the gradient of the input to the next hidden state
            da_next = np.dot(self.Wr[:, :self.hidden_size].T, dr) \
                      + np.dot(self.Wz[:, :self.hidden_size].T, dz) \
                      + np.dot(self.Wa[:, :self.hidden_size].T, da)
        # clip gradients to avoid exploding gradients
        for grad in [self.dWz, self.dWr, self.dWa, self.dWy, self.dbz, self.dbr, self.dba, self.dby]:
            np.clip(grad, -1, 1)

    # def loss(self, y_preds, targets):
    #     """
    #     Computes the cross-entropy loss for a given sequence of predicted probabilities and true targets.

    #     Parameters:
    #         y_preds (ndarray): Array of shape (sequence_length, vocab_size) containing the predicted probabilities for each time step.
    #         targets (ndarray): Array of shape (sequence_length, 1) containing the true targets for each time step.

    #     Returns:
    #         float: Cross-entropy loss.
    #     """
    #     # calculate cross-entropy loss

    #     result = sum(-np.log(y_preds[t][targets[t], 0]) for t in range(self.sequence_length))
    #     print('result:', result.shape)


    #     return sum(-np.log(y_preds[t][targets[t], 0]) for t in range(self.sequence_length))

    
    def loss(self, y_preds, targets, y_initIsc1, y_initIsc2, y_initIp, train_u2ss, train_basal, train_ins, train_meal):
        # har_data_instance = HarData()  # Create an instance of HarData

        # # Convert 3D array self.y_initIsc1 to 2D by taking the first slice along the first axis
        # self.y_initIsc1 = har_data_instance.train_initIsc1[0, :, :]
        # # print('y_initIsc1 (converted to 2D):', self.y_initIsc1.shape)

        # # Convert 3D array self.y_initIsc2 to 2D by taking the first slice along the first axis
        # self.y_initIsc2 = har_data_instance.train_initIsc2[0, :, :]
        # # print('y_initIsc2 (converted to 2D):', self.y_initIsc2.shape)

        # # Convert 3D array self.y_initIp to 2D by taking the first slice along the first axis
        # self.y_initIp = har_data_instance.train_initIp[0, :, :]
        # # print('y_initIp (converted to 2D):', self.y_initIp.shape)

        # # Convert 3D array self.train_u2ss to 2D by taking the first slice along the first axis
        # self.train_u2ss = har_data_instance.train_u2ss[0, :, :]
        # # print('train_u2ss (converted to 2D):', self.train_u2ss.shape)

        # # Convert 3D array self.train_basal to 2D by taking the first slice along the first axis
        # self.train_basal = har_data_instance.train_basal[0, :, :]
        # # print('train_basal (converted to 2D):', self.train_basal.shape)

        # # Convert 3D array self.train_ins to 2D by taking the first slice along the first axis
        # self.train_ins = har_data_instance.train_ins[0, :, :]
        # # print('train_ins (converted to 2D):', self.train_ins.shape)

        # # Convert 3D array self.train_meal to 2D by taking the first slice along the first axis
        # self.train_meal = har_data_instance.train_meal[0, :, :]
        # # print('train_meal (converted to 2D):', self.train_meal.shape)
        



        ############################# Parameters ##################################
        maxChange = 50
        kempt = (1 + (0.5 - y_preds[0]) * maxChange / 100) * 0.18
        kabs = (1 + (0.5 - y_preds[1]) * maxChange / 100) * 0.012
        f = 0.9
        Gb = (1 + (0.5 - y_preds[ 2]) * maxChange / 100) * 119.13
        SG = (1 + (0.5 - y_preds[3]) * maxChange / 100) * 0.025
        Vg = 1.45
        p2 = (1 + (0.5 - y_preds[4]) * maxChange / 100) * 0.012
        SI = (1 + (0.5 - y_preds[5]) * maxChange / 100) * 0.001035 / Vg
        Ipb = (1 + (0.5 - y_preds[6]) * maxChange / 100)
        alpha = 7
        kd = (1 + (0.5 - y_preds[7]) * maxChange / 100) * 0.026
        beta = np.floor((1 + (0.5 - y_preds[9]) * maxChange / 100) * 15)
        Vi = 0.126
        ka2 = (1 + (0.5 - y_preds[8]) * maxChange / 100) * 0.014
        ke = 0.127
        bolusD = np.floor((1 + (0.5 - y_preds[10]) * maxChange / 100) * 5)
        r2 = 0.8124

        # print("vals {}".format(kempt))
        # print("vals {}".format(kabs))
        # print("vals {}".format(Gb))
        # print("vals {}".format(SG))
        # print("vals {}".format(p2))

        ###########################################################################
        # Initialize GVal as a 1D array (from the first element of targets)
        target_array = np.array(targets)
        GVal = target_array            # 2D array from 3D, removed the third dimension
        # print('GVal:', GVal.shape)
        pmVal = np.zeros(GVal.shape, dtype=np.float32)
        Isc1Val = y_initIsc1[:, 0]      # Keep 2D, removed third dimension
        Isc2Val = y_initIsc2[:, 0]      # Keep 2D, removed third dimension
        IpVal = y_initIp[:, 0]          # Keep 2D, removed third dimension
        Qsto1Val = np.zeros(GVal.shape, dtype=np.float32)
        Ipb = np.zeros(GVal.shape, dtype=np.float32)
        Qsto2Val = np.zeros(GVal.shape, dtype=np.float32)
        QgutVal = np.zeros(GVal.shape, dtype=np.float32)
        RatVal = np.zeros(GVal.shape, dtype=np.float32)
        insVal = np.zeros(GVal.shape, dtype=np.float32)
        xVal = np.zeros(GVal.shape, dtype=np.float32)
        gVal = target_array            # 2D array from 3D, removed the third dimension
        y_ins = np.zeros(GVal.shape, dtype=np.float32)
        meal = np.zeros(GVal.shape, dtype=np.float32)

        # No need for np.expand_dims since everything is 1D now


        limitLoop = 43
        tau = 1
        stableEps = 100000.0

        kempt_expanded = np.tile(kd, (44 // 13 + 1, 1))[:44, :]
        p2_expanded = np.tile(kd, (44 // 13 + 1, 1))[:44, :]
        SI_expanded = np.tile(kd, (44 // 13 + 1, 1))[:44, :]

        for i in range(1, limitLoop):
            ka1 = 0.0
            
            # For 1D arrays, adjust the indexing and remove the second and third dimensions
            kd_expanded = np.tile(kd, (44 // 13 + 1, 1))[:44, :]
            # print('kd:', kd.shape)
            # print('ks:', ke)
            Ipb = (kd_expanded / ke * train_u2ss[i-1]) / kd_expanded
            # shape1 = GVal[i-1].shape

            # Update the logic to work with 1D arrays
            D = np.where((GVal[i-1] >= 60.0) & (GVal[i-1] < 119.13), 1.0, 0.0)
            E = np.where(GVal[i-1] < 60.0, 1.0, 0.0)
            
            risk = np.abs(
                10 * np.square(np.power(np.log(GVal[i - 1]), r2) - np.power(np.log(119.13), r2)) * D + 
                10 * np.power(np.power(np.log(60), r2) - np.power(np.log(119.13), r2), 2) * E
            )


            # Update all state variables to 1D
            dummyIsc1 = Isc1Val[i-1] + tau * (-kd_expanded * Isc1Val[i-1] + (train_basal[i-1] + train_ins[i-1]) / Vi)
            dummyIsc2 = Isc2Val[i-1] + tau * (kd * Isc1Val[i-1] - ka2 * Isc2Val[i-1])
            dummyIp = IpVal[i-1] + tau * (ka2 * Isc2Val[i-1] - ke * IpVal[i-1])
            dummyQsto1 = Qsto1Val[i-1] + tau * (-kempt_expanded * Qsto1Val[i-1] + train_meal[i-1])
            dummyQsto2 = Qsto2Val[i-1] + tau * (kempt * Qsto1Val[i-1] - kempt * Qsto2Val[i-1])
            dummyQgut = QgutVal[i-1] + tau * (kempt * Qsto2Val[i-1] - kabs * QgutVal[i-1])
            
            RatVal = f * kabs * QgutVal[i-1]
            dummyXVal = xVal[i-1] + tau * (-p2_expanded * xVal[i-1] - SI_expanded * (IpVal[i-1] - Ipb))
            dummygVal = gVal[i-1] + tau * (-(SG + risk * xVal[i-1]) * gVal[i-1] + SG * Gb + RatVal / Vg)

            dummyG1 = GVal[i-1] + tau * (-(1 / alpha) * (GVal[i-1] - gVal[i-1]))
            diffDummy = dummyG1 - GVal[i-1]
            sumDiff = np.sum(np.square(diffDummy)) / 256 - stableEps
            dummyG1 = np.where(sumDiff > 0.0, GVal[i-1], dummyG1)

            # Append new values to the corresponding arrays
            Isc1Val = np.append(Isc1Val, dummyIsc1)
            Isc2Val = np.append(Isc2Val, dummyIsc2)
            IpVal = np.append(IpVal, dummyIp)
            Qsto1Val = np.append(Qsto1Val, dummyQsto1)
            Qsto2Val = np.append(Qsto2Val, dummyQsto2)
            QgutVal = np.append(QgutVal, dummyQgut)
            gVal = np.append(gVal, dummygVal)
            xVal = np.append(xVal, dummyXVal)
            GVal = np.append(GVal, dummyG1)

        # Calculate error over the loop
        err = np.sqrt(np.mean(np.square(targets[limitLoop] - GVal)))
        print('err:', err)

#         # Break the loop and return the error if it's below the threshold
#         if err < 3.2:
#             print('err:', err)
#             return err

        return err




    


    def adamw(self, beta1=0.9, beta2=0.999, epsilon=1e-8, L2_reg=1e-4):
        """
        Updates the GRU's parameters using the AdamW optimization algorithm.
        """
        
        # AdamW update for Wz
        self.mWz = beta1 * self.mWz + (1 - beta1) * self.dWz
        self.vWz = beta2 * self.vWz + (1 - beta2) * np.square(self.dWz)
        m_hat = self.mWz / (1 - beta1)
        v_hat = self.vWz / (1 - beta2)
        self.Wz -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wz)

        # AdamW update for bu
        self.mbz = beta1 * self.mbz + (1 - beta1) * self.dbz
        self.vbz = beta2 * self.vbz + (1 - beta2) * np.square(self.dbz)
        m_hat = self.mbz / (1 - beta1)
        v_hat = self.vbz / (1 - beta2)
        self.bz -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.bz)

        # AdamW update for Wr
        self.mWr = beta1 * self.mWr + (1 - beta1) * self.dWr
        self.vWr = beta2 * self.vWr + (1 - beta2) * np.square(self.dWr)
        m_hat = self.mWr / (1 - beta1)
        v_hat = self.vWr / (1 - beta2)
        self.Wr -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wr)

        # AdamW update for br
        self.mbr = beta1 * self.mbr + (1 - beta1) * self.dbr
        self.vbr = beta2 * self.vbr + (1 - beta2) * np.square(self.dbr)
        m_hat = self.mbr / (1 - beta1)
        v_hat = self.vbr / (1 - beta2)
        self.br -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.br)

        # AdamW update for Wa
        self.mWa = beta1 * self.mWa + (1 - beta1) * self.dWa
        self.vWa = beta2 * self.vWa + (1 - beta2) * np.square(self.dWa)
        m_hat = self.mWa / (1 - beta1)
        v_hat = self.vWa / (1 - beta2)
        self.Wa -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wa)

        # AdamW update for br
        self.mba = beta1 * self.mba + (1 - beta1) * self.dba
        self.vba = beta2 * self.vba + (1 - beta2) * np.square(self.dba)
        m_hat = self.mba / (1 - beta1)
        v_hat = self.vba / (1 - beta2)
        self.ba -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.ba)

        # AdamW update for Wy
        self.mWy = beta1 * self.mWy + (1 - beta1) * self.dWy
        self.vWy = beta2 * self.vWy + (1 - beta2) * np.square(self.dWy)
        m_hat = self.mWy / (1 - beta1)
        v_hat = self.vWy / (1 - beta2)
        self.Wy -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wy)

        # AdamW update for by
        self.mby = beta1 * self.mby + (1 - beta1) * self.dby
        self.vby = beta2 * self.vby + (1 - beta2) * np.square(self.dby)
        m_hat = self.mby / (1 - beta1)
        v_hat = self.vby / (1 - beta2)
        self.by -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.by)

    def train(self, data_generator,iterations):
        """
        Train the GRU on a dataset using backpropagation through time.

        Args:
            data_generator: An instance of DataGenerator containing the training data.

        Returns:
            None
        """
        iter_num = 0
        # stopping criterion for training
        threshold = 50
    
        smooth_loss = -np.log(1.0 / data_generator.vocab_size) * self.sequence_length  # initialize loss

        har_data_instance = HarData()  # Create an instance of HarData

        # Convert 3D array self.y_initIsc1 to 2D by taking the first slice along the first axis
        self.y_initIsc1 = har_data_instance.train_initIsc1[0, :, :]
        # print('y_initIsc1 (converted to 2D):', self.y_initIsc1.shape)

        # Convert 3D array self.y_initIsc2 to 2D by taking the first slice along the first axis
        self.y_initIsc2 = har_data_instance.train_initIsc2[0, :, :]
        # print('y_initIsc2 (converted to 2D):', self.y_initIsc2.shape)

        # Convert 3D array self.y_initIp to 2D by taking the first slice along the first axis
        self.y_initIp = har_data_instance.train_initIp[0, :, :]
        # print('y_initIp (converted to 2D):', self.y_initIp.shape)

        # Convert 3D array self.train_u2ss to 2D by taking the first slice along the first axis
        self.train_u2ss = har_data_instance.train_u2ss[0, :, :]
        # print('train_u2ss (converted to 2D):', self.train_u2ss.shape)

        # Convert 3D array self.train_basal to 2D by taking the first slice along the first axis
        self.train_basal = har_data_instance.train_basal[0, :, :]
        # print('train_basal (converted to 2D):', self.train_basal.shape)

        # Convert 3D array self.train_ins to 2D by taking the first slice along the first axis
        self.train_ins = har_data_instance.train_ins[0, :, :]
        # print('train_ins (converted to 2D):', self.train_ins.shape)

        # Convert 3D array self.train_meal to 2D by taking the first slice along the first axis
        self.train_meal = har_data_instance.train_meal[0, :, :]
        # print('train_meal (converted to 2D):', self.train_meal.shape)
        
        early_stop_patience = 3  # The number of consecutive iterations with small loss change
        consecutive_small_loss_change = 0  # Counter for consecutive small loss changes
        threshold = 0.10  # 10% threshold for loss change
        prev_loss = None  # To store the previous loss value

        while (iter_num < iterations):
            # initialize hidden state at the beginning of each sequence
            if data_generator.pointer == 0:
                c_prev = np.zeros((self.hidden_size, 1))
                a_prev = np.zeros((self.hidden_size, 1))

            # get a batch of inputs and targets
            inputs, targets = data_generator.next_batch()

            # forward pass
            X, r, z, c, cc, a, y_pred = self.forward(inputs, c_prev, a_prev)
            # print('y_pred_list:', len(y_pred))
            # print('y_pred_list:', y_pred[199].shape)

            # backward pass
            self.backward(X, a_prev, c_prev, r, z, c, cc, a, y_pred, targets)

            # calculate and update loss
#             loss = self.loss(y_pred, targets, self.y_initIsc1, self.y_initIsc2, self.y_initIp, self.train_u2ss, self.train_basal, self.train_ins, self.train_meal)
            # print('loss shape:', loss)
            # print('loss shape:')
            
            loss =  np.sqrt(np.mean(np.square(np.array(list(y_pred.values())) - np.array(targets))))
            print('loss:', loss)
            
            
#              # Early stopping logic
#             if prev_loss is not None:
#                 # Calculate the percentage change in loss
#                 loss_diff = abs(prev_loss - loss) / prev_loss

#                 if loss_diff < threshold:  # If the loss change is less than 10%
#                     consecutive_small_loss_change += 1
#                     if consecutive_small_loss_change >= early_stop_patience:
#                         print(f"Early stopping triggered at iteration {iter_num} with loss: {loss}")
#                         break
#                 else:
#                     consecutive_small_loss_change = 0  # Reset the counter if the change is larger than 10%

#             prev_loss = loss  # Update the previous loss
            
            self.adamw()
            smooth_loss = smooth_loss * 0.999 + loss * 0.001
            # update previous hidden state for the next batch
            a_prev = a[self.sequence_length - 1]
            c_prev = c[self.sequence_length - 1]
#             if iter_num == 5900 or iter_num == 30000:
#                         self.learning_rate *= 0.1
            # print progress every 100 iterations
            if iter_num % 100 == 0:
#                 self.learning_rate *= 0.99
                sample_idx = self.sample(c_prev, a_prev, inputs[0, :], 200)
                print(''.join(data_generator.idx_to_char[idx] for idx in sample_idx))
                print("\n\niter :%d, loss:%f" % (iter_num, smooth_loss))
            iter_num += 1
            
        return smooth_loss, y_pred

    def sample(self, c_prev, a_prev, seed_idx, n):
        """
        Sample a sequence of integers from the model.

        Args:
            c_prev (numpy.ndarray): Previous cell state, a numpy array of shape (hidden_size, 1).
            a_prev (numpy.ndarray): Previous hidden state, a numpy array of shape (hidden_size, 1).
            seed_idx (numpy.ndarray): Seed letter from the first time step, a numpy array of shape (vocab_size, 1).
            n (int): Number of characters to generate.

        Returns:
            list: A list of integers representing the generated sequence.

        """
        # initialize input and seed_idx
        x = np.zeros((self.vocab_size, 1))
        # convert one-hot encoding to integer index
        seed_idx = np.argmax(seed_idx, axis=-1)

        # set the seed letter as the input for the first time step
        x[seed_idx] = 1

        # generate sequence of characters
        idxes = []
        c = np.copy(c_prev)
        a = np.copy(a_prev)
        for t in range(n):
            # compute the hidden state and cell state
            concat = np.vstack((a, x))
            z = self.sigmoid(np.dot(self.Wz, concat) + self.bz)
            r = self.sigmoid(np.dot(self.Wr, concat) + self.br)
            cc = np.tanh(np.dot(self.Wa, np.vstack((r * a, x))) + self.ba)
            c = z * c + (1 - z) * cc
            a = c
            # compute the output probabilities
            y = self.softmax(np.dot(self.Wy, a) + self.by)

            # sample the next character from the output probabilities
            idx = np.random.choice(range(self.vocab_size), p=y.ravel())

            # set the input for the next time step
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1

            # append the sampled character to the sequence
            idxes.append(idx)

        # return the generated sequence
        return idxes

    def predict(self, data_generator, start, n):
        """
        Generate a sequence of n characters using the trained GRU model, starting from the given start sequence.

        Args:
        - data_generator: an instance of DataGenerator
        - start: a string containing the start sequence
        - n: an integer indicating the length of the generated sequence

        Returns:
        - txt: a string containing the generated sequence
        """
        # initialize input sequence
        x = np.zeros((self.vocab_size, 1))
        chars = [ch for ch in start]
        idxes = []
        for i in range(len(chars)):
            idx = data_generator.char_to_idx[chars[i]]
            # idx = data_generator[chars[i]]
            x[idx] = 1
            idxes.append(idx)
        # initialize cell state and hidden state
        a = np.zeros((self.hidden_size, 1))
        c = np.zeros((self.hidden_size, 1))

        # generate new sequence of characters
        for t in range(n):
            # compute the hidden state and cell state
            concat = np.vstack((a, x))

            # compute the reset gate
            r = self.sigmoid(np.dot(self.Wr, concat) + self.br)

            # compute the update gate
            z = self.sigmoid(np.dot(self.Wz, concat) + self.bz)

            # compute the candidate cell state
            cc = np.tanh(np.dot(self.Wa, np.vstack((r * a, x))) + self.ba)

            # compute the cell state
            c = z * cc + (1 - z) * c

            # compute the hidden state
            a = c

            # compute the output probability vector
            y_pred = self.softmax(np.dot(self.Wy, a) + self.by)
            # sample the next character from the output probabilities
            idx = np.random.choice(range(self.vocab_size), p=y_pred.ravel())
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1
            idxes.append(idx)
        txt = ''.join(data_generator.idx_to_char[i] for i in idxes)
        return txt


In [ ]:
import psutil
import time
import subprocess

def print_memory_usage():
    """Function to print current memory and CPU usage"""
    memory_info = psutil.virtual_memory()
    cpu_usage = psutil.cpu_percent(interval=1)
    
    print(f"Total memory: {memory_info.total / (1024 ** 2):.2f} MB")
    print(f"Available memory: {memory_info.available / (1024 ** 2):.2f} MB")
    print(f"Used memory: {memory_info.used / (1024 ** 2):.2f} MB")
    print(f"Memory percent used: {memory_info.percent}%")
    print(f"CPU usage: {cpu_usage}%\n")
    
def print_gpu_usage():
    """Function to print current GPU power usage (if using a GPU)"""
    try:
        gpu_power_usage = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=power.draw", "--format=csv,noheader,nounits"]
        )
        print(f"GPU Power usage: {gpu_power_usage.decode('utf-8').strip()} W")
    except Exception as e:
        print(f"Could not fetch GPU power usage: {e}")

# Print memory usage before buffer allocation
print("Memory usage before allocation:")
print_memory_usage()

# Print GPU power usage if available
print_gpu_usage()

sequence_length = 50
# Read text from the "input.txt" file
data_generator = DataGenerator('Train_570_13_glucose.txt', sequence_length)
gru = GRU(hidden_size=128, vocab_size=data_generator.vocab_size, sequence_length=sequence_length, learning_rate=0.001)

start_time = time.time()

smooth_loss, y_pred = gru.train(data_generator, iterations=32)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Training took {elapsed_time:.2f} seconds")

# Print memory usage after training
print("Memory usage after training:")
print_memory_usage()

# Print GPU power usage again after training
print_gpu_usage()
